<a href="https://colab.research.google.com/github/thegreatgupta/ObjectRecognition/blob/master/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [4]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.864645031000236
GPU (s):
0.1335776689998056
GPU speedup over CPU: 66x


In [0]:
import tarfile

In [0]:
import os

In [0]:
tar = tarfile.open("object-classification-dataset.tar")

In [0]:
tar.extractall()

In [9]:
print(tar)

In [0]:
tar.close()

In [11]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
def model_Xception(img_shape=(299, 299, 3), n_classes=50,
                   load_pretrained=False, freeze_layers_from='base_model'):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = 'imagenet'
    else:
        weights = None

    # Get base model
    #base_model = Xception(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    base_model = ResNet50(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)
    
    base_model = MobileNet(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)

    # Add final layers
    x = base_model.output
    x = Dense(n_classes, activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(n_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(input=base_model.input, output=predictions)
    
    # Model Summary
    # print(model.summary())
    
    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == 'base_model':
            print ('   Freezing base model layers')
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
               layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
               layer.trainable = True

    print(model.summary())
    return model

In [0]:
def train_Xception(train_set_path, val_set_path, validation_split = 0.2,
                   batch_size = 32, class_mode = 'categorical', horizontal_flip = False,
                   vertical_flip = False, rotation_range = None, target_size = (299, 299),
                   model = None, epochs = 1):
    
    # Load Image Data Set Using Keras
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = horizontal_flip,
                                   vertical_flip = vertical_flip,
                                   rotation_range=rotation_range,
                                   validation_split=validation_split)
    
    training_set = train_datagen.flow_from_directory(train_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'training')
    
    validation_set = train_datagen.flow_from_directory(val_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'validation')
    
    
    # Compile Model
    learning_rate = 0.0001
    loss = 'categorical_crossentropy'
    # opt_rms = keras.optimizers.rmsprop(lr=learning_rate,decay=1e-6)
    adam_optimizer = Adam(lr = learning_rate)
    model.compile(optimizer = adam_optimizer, loss = loss, metrics = ['accuracy'])
    
    model.fit_generator(training_set,
                        steps_per_epoch = training_set.samples,
                        epochs = epochs,
                        validation_data = validation_set,
                        validation_steps = validation_set.samples)
    
    
    return model

In [0]:
# Declare Constants
img_shape = (100, 100, 3)
target_size = (100, 100)
n_classes = 49
epochs = 40
train_set_path = '101_ObjectCategories' #'dataset2/training_set_test'
val_set_path = train_set_path
validation_split = 0.15
batch_size = 8
horizontal_flip = True
rotation_range = None
learning_rate = 0.0001
loss = 'categorical_crossentropy'


In [27]:
# Build Model
model = model_Xception(img_shape = img_shape,  n_classes = n_classes, load_pretrained = True)

   Freezing base model layers
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [28]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/40
4584/4584 [==============================] - 245s 53ms/step - loss: 2.8268 - acc: 0.3945 - val_loss: 1.8115 - val_acc: 0.5713
Epoch 2/40
4584/4584 [==============================] - 241s 53ms/step - loss: 2.0164 - acc: 0.5419 - val_loss: 1.6776 - val_acc: 0.6282
Epoch 3/40
4584/4584 [==============================] - 244s 53ms/step - loss: 1.7425 - acc: 0.5867 - val_loss: 1.6617 - val_acc: 0.6497
Epoch 4/40
4584/4584 [==============================] - 244s 53ms/step - loss: 1.5817 - acc: 0.6193 - val_loss: 1.7322 - val_acc: 0.6513
Epoch 5/40
4584/4584 [==============================] - 244s 53ms/step - loss: 1.4867 - acc: 0.6389 - val_loss: 1.7056 - val_acc: 0.6612
Epoch 6/40
4584/4584 [==============================] - 241s 53ms/step - loss: 1.4169 - acc: 0.6501 - val_loss: 1.6451 - val_acc: 0.6679
Epoch 7/40
4584/4584 [==============================] - 243s 53ms/step - loss: 1.3530 - acc: 

In [0]:
model.save_weights('obj_recognition_xception_normal_ep1.h5')